In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='2'
#import wget
import sys
sys.path.append('../')
import argparse
import json
import pandas as pd
import random
import numpy as np
import string
import nltk
from functools import partial
import re
from  tqdm import tqdm
import torch
nltk.download('punkt')
from src.dataset_classes import DatasetObject,Features,SquadQuestionGenerationDataset
from src.utils import answerGeneratorDataset,questionGeneratorDataset,buildFact,setuptokenizer,pad_seq,SmartCollator
from dataclasses import dataclass, field
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
model_base = 'facebook/bart-base'
tokenizer = setuptokenizer(model_base=model_base,
                           special_tokens=['<section>','</section>'
                                           ,'<generate_questions>',
                                           '<generate_answers>'])

In [3]:
dataset = SquadQuestionGenerationDataset(tokenizer,nb_records=1)
dataset.change_data_mode(1)

In [4]:
from transformers import BartForConditionalGeneration
# Set up the model
def model_init(device =  torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') ):
    generator = BartForConditionalGeneration.from_pretrained(model_base)
    # update the tokens 
    generator.resize_token_embeddings(len(tokenizer))
    return generator.to(device)

trained_weights = torch.load('../trained_models/bart_base_model_1/checkpoint-8000/pytorch_model.bin')

generator = model_init()
generator.load_state_dict(trained_weights)
device = generator.device

In [5]:
from nltk.util import ngrams
import wikipedia
def factgenerator(document,n):
    return list(ngrams(sent_tokenize(document.strip()),n))

In [6]:
sample_too = True
sampling_helper = {} if not sample_too else dict(top_k=30, top_p=0.95,)
max_length=250
length_penalty=2.6
beam_size=4
repetition_penalty=1.56
return_top_beams= beam_size if not sample_too else 10

In [7]:
article = wikipedia.summary('Renewable Energy')
n=4
facts = [' '.join(s).replace('\n','').strip() for s in factgenerator(article.replace('.T','. T'),n=n)]

In [17]:
from  mosestokenizer.detokenizer import MosesDetokenizer

In [22]:
tokenize = MosesDetokenizer('en')

In [21]:
tokenize('White is an achromatic color , a color without hue .'.split())

'White is an achromatic color, a color without hue.'

In [11]:
task_id = 0
target_fact = facts[8]

#' item[COA], volume[32m3],  ratio[0.06]  Table 1: Chemical combination rule for working with N2 gas.'
data = DatasetObject(task='<generate_questions> ', question='',
                     context=target_fact, fact=target_fact,
                     answer='',
                     answer_sentence='',
                     task_id="")

batch = dataset.procesTexts(data)

b_input_ids = batch.input_ids.view(1, -1).to(device)
b_input_mask = batch.attention_mask.view(1, -1).to(device)

sample_too = True
sampling_helper = {} if not sample_too else dict(top_k=25, top_p=0.95,)
return_top_beams = beam_size if not sample_too else 25
# seed_everything(2982)
generator.eval()
with torch.no_grad():
    sample_outputs = generator.generate(input_ids=b_input_ids,  **sampling_helper,
                                        attention_mask=b_input_mask,
                                        num_beams=beam_size,
                                        repetition_penalty=repetition_penalty,
                                        length_penalty=length_penalty,
                                        early_stopping=False,
                                        use_cache=True,
                                        max_length=max_length,
                                        no_repeat_ngram_size=2,
                                        num_return_sequences=return_top_beams,
                                        do_sample=sample_too,
                                        eos_token_id=dataset.tokenizer.eos_token_id,)
oop = [dataset.tokenizer.decode(sample_outputs[idx],
                                skip_special_tokens=True,
                                clean_up_tokenization_spaces=True) for idx in range(return_top_beams)]

print(f'Article Section: {data.context}')
print('Questions Generated')
for q in set(oop):
    print(q)


Article Section: Biomass and geothermal energy grew from 2% to 3%. There are 3,146 gigawatts installed in 135 countries, while 156 countries have laws regulating the renewable energy sector. Globally there are over 10 million jobs associated with the renewable energy industries, with solar photovoltaics being the largest renewable employer. Renewable energy systems are rapidly becoming more efficient and cheaper and their share of total energy consumption is increasing, with a large majority of worldwide newly installed electricity capacity being renewable.
Questions Generated
What percentage of electricity capacity is renewable?
How many gigawatts of renewable energy capacity are in 135 countries?
What percentage of electricity capacity in the world is solar photovoltaics?
What is the percentage of installed electricity capacity being renewable?
How many gigawatts of solar photovoltaics are there in the world?
What percentage of worldwide newly installed electricity capacity is renewa

In [21]:
for q in oop:
    print(q)

What is the name of the pancake that is made with buttermilk in place of eggs?
What is the name of the type of pancake made with buttermilk in place of eggs?
What is the pancake with buttermilk in place of or in addition to milk known as in Scotland and the US?
What is the name of the pancake that is made with buttermilk in place of eggs when it is used in addition to milk?
What is the name of the pancake that is used in place of buttermilk in addition to milk?
What is the name of the pancake that is used in place of buttermilk in addition to milk?
What is the name of the type of pancake that is made with buttermilk in place of eggs when it is used in addition to milk?
What is the name of the pancake that is used in place of buttermilk when it is not used?
What type of pancake can be used in place of buttermilk to make it tart?
What is a type of buckwheat pancake that can be used in place of buttermilk or in addition to milk?
What is the name of the pancake that is used when buttermilk